## Automated Extraction of Management Discussion & Analysis (MD&A) Sections from Indian Annual Report PDFs

### Introduction

The Management Discussion & Analysis (MD&A) section is a critical component of corporate annual reports, providing qualitative insights into a company's financial performance, operational challenges, risk factors, and future outlook. As mandated by regulatory frameworks such as the Companies Act, 2013 in India, MD&A serves as a strategic tool for stakeholders to assess management perspectives beyond quantitative financial statements, enabling informed decision-making in investment, risk assessment, and corporate governance.

Extracting MD&A content from PDF-based annual reports presents significant technical challenges. Annual reports are inherently unstructured documents, featuring complex layouts with embedded tables, images, and multi-column text that complicate text extraction. Layout variability across companies due to differing design choices, font styles, and page structures further hinders automated processing. Additionally, MD&A sections are often integrated with other report components, such as Directors' Reports or financial statements, making precise boundary identification difficult.

Indian annual reports exhibit particular structural diversity in MD&A presentation. Some companies provide standalone MD&A sections, while others embed the content within annexures or integrate it directly into the Directors' Report. This variability necessitates robust extraction methods capable of adapting to multiple organizational patterns.

This notebook implements a systematic pipeline for MD&A extraction, comprising the following stages:

1. **Data Collection**: Identification and organization of PDF annual reports from diverse Indian companies.
2. **PDF Parsing**: Extraction of raw text and structural elements using specialized libraries.
3. **Text Preprocessing**: Cleaning and normalization of extracted content to handle encoding artifacts and formatting inconsistencies.
4. **Section Detection**: Identification of MD&A boundaries through pattern matching and keyword-based analysis.
5. **Content Extraction**: Precise isolation of MD&A text while filtering extraneous sections.
6. **Validation and Output**: Quality assessment of extracted content and structured output generation for downstream analysis.

### 2. Imports & Configuration Layer : 

In [27]:
import fitz  
import pandas as pd
import re
import logging
from tqdm import tqdm
import pathlib

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

PROJECT_ROOT = pathlib.Path.cwd()

INPUT_PDF_DIR = PROJECT_ROOT / "../data" / "../pdfs"
OUTPUT_DIR = PROJECT_ROOT / "../output"

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

### 3. PDF Interface Layer : 

In [21]:
class PDFInterface:
    def __init__(self, pdf_path):
        self.pdf_path = pathlib.Path(pdf_path)
        self.doc = fitz.open(self.pdf_path)
        logging.info("Loaded PDF: %s", self.pdf_path.name)

    def get_pages_text(self):
        pages = []
        for page_index in range(self.doc.page_count):
            page = self.doc.load_page(page_index)
            pages.append(
                {
                    "page_number": page_index + 1,
                    "text": page.get_text(),
                }
            )
        return pages

    def close(self):
        self.doc.close()

###  4. Company Name & Financial Year Extraction: 

In [58]:
def extract_company_name(pages_text):
    """
    Extract company name from the first 3 pages of PDF text.

    Args:
        pages_text: List of page dictionaries with 'page_number' and 'text'

    Returns:
        str: Company name or None if not found
    """
    # Regex pattern for company names ending with LIMITED, Limited, or Ltd.
    company_pattern = re.compile(
        r'([A-Z][A-Za-z\s&\(\),.-]+?(?:LIMITED|Limited|Ltd\.?))',
        re.MULTILINE,
    )

    matches = []

    # Search only first 3 pages
    for page in pages_text[:3]:
        text = page['text']
        found = company_pattern.findall(text)
        matches.extend(found)

    if not matches:
        logging.info("Company name not found in first 3 pages")
        return None

    # Return the longest match (most confident)
    company_name = max(matches, key=len).strip()
    logging.info("Extracted company name: %s", company_name)
    return company_name


def extract_financial_year(pages_text):
    """Extract financial year from the first 5 pages of PDF text.

    Supported examples:
      - "Annual Report 2019-20"
      - "31st Annual Report 2019-20"
      - "Year ended March 31, 2020"

    Returns:
        str: Normalized financial year (e.g., '2019-20') or None if not found
    """

    # Patterns for various year formats (search order matters: more specific first)
    year_patterns = [
        # 31st Annual Report 2019-20 / 31st Annual Report 2019 - 2020
        re.compile(
            r"\b\d{1,3}(?:st|nd|rd|th)\s+Annual\s+Report\s+(\d{4})\s*[-–]\s*(\d{2,4})\b",
            re.IGNORECASE,
        ),
        # Annual Report 2019-20 / Annual Report 2019 - 2020
        re.compile(
            r"\bAnnual\s+Report\s+(\d{4})\s*[-–]\s*(\d{2,4})\b",
            re.IGNORECASE,
        ),
        # Year ended March 31, 2020 (or similar)
        re.compile(
            r"\bYear\s+ended\s+\w+\s+\d{1,2},\s+(\d{4})\b",
            re.IGNORECASE,
        ),
    ]

    # Search first 5 pages
    for page in pages_text[:5]:
        text = page.get('text', '')

        for pattern in year_patterns:
            match = pattern.search(text)
            if not match:
                continue

            groups = match.groups()

            if len(groups) == 1:
                # Single year (e.g., Year ended March 31, 2020) -> previous year - last 2 digits
                year = groups[0]
                prev_year = str(int(year) - 1)
                normalized = f"{prev_year}-{year[-2:]}"
            else:
                year1, year2 = groups
                if len(year2) == 2:
                    normalized = f"{year1}-{year2}"
                else:
                    normalized = f"{year1}-{year2[-2:]}"

            logging.info("Extracted financial year: %s", normalized)
            return normalized

    logging.info("Financial year not found in first 5 pages")
    return None


### 5. Table of Contents (ToC) Analyzer: 

In [85]:
def collect_toc_raw_lines(pages_text, max_pages=5):
    """Collect STRICT ToC raw lines from ONLY the first `max_pages` pages.

    Returns a list preserving original extracted order exactly (per splitlines()).
    Each item:
      {
        "toc_page": int,
        "line_index": int,   # 0-based across non-empty ToC lines
        "line_text": str
      }

    Notes:
      - This collects ALL non-empty lines (not just those with digits), because
        MD&A titles can appear without a page number on the same extracted line,
        and titles can be wrapped across multiple extracted lines.
      - Downstream logic MUST NOT consult body text; this is ToC-page-only.
    """

    raw_lines = []
    line_index = 0

    for page in pages_text[:max_pages]:
        toc_page_no = page.get("page_number")
        text = page.get("text", "") or ""

        for ln in text.splitlines():
            s = (ln or "").strip()
            if not s:
                continue

            raw_lines.append(
                {
                    "toc_page": toc_page_no,
                    "line_index": line_index,
                    "line_text": s,
                }
            )
            line_index += 1

    return raw_lines


def detect_toc_entries(pages_text, max_pages=5):
    """STRICT ToC line collection.

    Non-negotiable behavior (per spec):
      - Scan ONLY the first `max_pages` pages.
      - Collect all non-empty lines containing BOTH:
          * at least one alphabetic character, AND
          * at least one numeric character
      - Preserve original line order exactly as extracted.

    Returns:
      list[dict] with keys: toc_page, line_index, line_text
    """

    raw_lines = collect_toc_raw_lines(pages_text, max_pages=max_pages)

    entries = []
    for item in raw_lines:
        txt = item.get("line_text", "")
        if re.search(r"[A-Za-z]", txt) and re.search(r"\d", txt):
            entries.append(item)

    logging.info("Collected %d strict ToC declaration lines (first %d pages)", len(entries), max_pages)
    return entries


_PAGE_INT_RE = re.compile(r"\b(\d{1,4})\b")


def _first_valid_page_number_in_text(text: str, max_page: int):
    for m in _PAGE_INT_RE.finditer(text or ""):
        try:
            n = int(m.group(1))
        except ValueError:
            continue
        if 1 <= n <= max_page:
            return n
    return None


def _first_valid_page_number_after_pos(text: str, start_pos: int, max_page: int):
    res = _first_valid_page_number_after_pos_with_span(text, start_pos, max_page)
    return res[0] if res else None


def _first_valid_page_number_after_pos_with_span(text: str, start_pos: int, max_page: int):
    for m in _PAGE_INT_RE.finditer(text or ""):
        if m.start() < (start_pos or 0):
            continue
        try:
            n = int(m.group(1))
        except ValueError:
            continue
        if 1 <= n <= max_page:
            return n, m.start(), m.end()
    return None


def resolve_page_number_strict(raw_lines, title_line_index: int, max_page: int, lookahead_lines: int = 3):
    """STRICT page-number association (no fallback logic).

    Rules:
      - If the title line contains a valid integer page number, use it.
      - ELSE look ONLY at the immediately following lines (max next `lookahead_lines`).
      - The FIRST valid integer page number encountered is used.
      - If none found in this strict window, return None.

    Note:
      - This is the generic helper; for section titles that may share a line with
        other sections (multi-column extraction), prefer
        resolve_page_number_for_title_block_strict().
    """

    if title_line_index < 0 or title_line_index >= len(raw_lines):
        return None

    # If the line contains a page number, use it.
    same_line = raw_lines[title_line_index].get("line_text", "")
    n = _first_valid_page_number_in_text(same_line, max_page)
    if n is not None:
        return n

    # Else look at the next lines only.
    for offset in range(1, lookahead_lines + 1):
        j = title_line_index + offset
        if j >= len(raw_lines):
            break

        candidate_line = raw_lines[j].get("line_text", "")
        n = _first_valid_page_number_in_text(candidate_line, max_page)
        if n is not None:
            return n

    return None


def find_title_block_strict(
    raw_lines,
    title_re: re.Pattern,
    max_join_lines: int = 3,
    anchor_re: re.Pattern | None = None,
):
    """Find a title match treating the ToC as structural blocks.

    Deterministic behavior:
      - Scans raw_lines in order.
      - Only considers a block starting at line i if anchor_re matches line i
        (when anchor_re is provided). This prevents accidentally starting a
        block on an unrelated neighboring section.
      - At each valid start position i, tests the concatenation of
        1..max_join_lines lines (joined with a single space) against title_re.
      - Returns (start_idx, end_idx, block_text) for the first match.

    It does NOT consult body text and does NOT search beyond the ToC pages.
    """

    n = len(raw_lines)
    for i in range(n):
        start_line = raw_lines[i].get("line_text", "")
        if anchor_re is not None and not anchor_re.search(start_line or ""):
            continue

        parts = []
        for j in range(i, min(n, i + max_join_lines)):
            parts.append(raw_lines[j].get("line_text", ""))
            block_text = " ".join(p for p in parts if p)
            if title_re.search(block_text or ""):
                return i, j, block_text

    return None, None, None


def resolve_page_number_for_title_block_strict(
    raw_lines,
    title_start_idx: int,
    title_end_idx: int,
    title_re: re.Pattern,
    max_page: int,
    lookahead_lines: int = 3,
):
    """STRICT page-number association for a title block, robust to multi-column merges.

    Deterministic rules:
      - If the title appears on a line that also contains multiple page numbers,
        choose the FIRST valid integer page number that occurs AFTER the matched
        title text on that line.
      - Otherwise (no resolvable number on the title-containing line), search
        ONLY the next `lookahead_lines` lines after the title block; FIRST valid
        integer page number wins.
      - If none found, return None.

    This is still ToC-only and bounded; no body-text inference.
    """

    details = resolve_page_number_for_title_block_strict_with_details(
        raw_lines,
        title_start_idx=title_start_idx,
        title_end_idx=title_end_idx,
        title_re=title_re,
        max_page=max_page,
        lookahead_lines=lookahead_lines,
    )
    return details["page"] if details else None


def resolve_page_number_for_title_block_strict_with_details(
    raw_lines,
    title_start_idx: int,
    title_end_idx: int,
    title_re: re.Pattern,
    max_page: int,
    lookahead_lines: int = 3,
):
    """Same as resolve_page_number_for_title_block_strict, but returns details.

    Returns dict:
      {
        "page": int,
        "page_span_start": int | None,
        "page_span_end": int | None,
        "page_line_idx": int,
      }
    """

    if title_start_idx is None or title_end_idx is None:
        return None

    # 1) Prefer a page number that occurs after the matched title on the same line.
    for i in range(title_start_idx, title_end_idx + 1):
        line = raw_lines[i].get("line_text", "")
        m = title_re.search(line or "")
        if not m:
            continue

        res = _first_valid_page_number_after_pos_with_span(line, m.end(), max_page)
        if res is not None:
            n, s, e = res
            return {
                "page": n,
                "page_span_start": s,
                "page_span_end": e,
                "page_line_idx": i,
            }

    # 2) Otherwise, search next N lines after the title block.
    for offset in range(1, lookahead_lines + 1):
        j = title_end_idx + offset
        if j >= len(raw_lines):
            break

        line = raw_lines[j].get("line_text", "")
        res = _first_valid_page_number_after_pos_with_span(line, 0, max_page)
        if res is not None:
            n, s, e = res
            return {
                "page": n,
                "page_span_start": s,
                "page_span_end": e,
                "page_line_idx": j,
            }

    return None


def find_mdna_start_from_toc(pages_text):
    """Backward-compatible helper: STRICT MD&A start-page discovery from ToC pages only."""

    raw_lines = collect_toc_raw_lines(pages_text, max_pages=5)
    if not raw_lines:
        logging.info("No ToC raw lines found in first 5 pages")
        return None

    max_page = len(pages_text)

    mdna_title_re = re.compile(
        r"\bmanagement\s+discussion\s+(?:and|&)\s+analysis(?:\s+report)?\b",
        re.IGNORECASE,
    )

    mdna_anchor_re = re.compile(r"\bmanagement\b", re.IGNORECASE)

    start_idx, end_idx, _ = find_title_block_strict(
        raw_lines,
        mdna_title_re,
        max_join_lines=3,
        anchor_re=mdna_anchor_re,
    )

    if start_idx is None:
        logging.info("MD&A title block not found in ToC raw lines")
        return None

    start_page = resolve_page_number_for_title_block_strict(
        raw_lines,
        title_start_idx=start_idx,
        title_end_idx=end_idx,
        title_re=mdna_title_re,
        max_page=max_page,
        lookahead_lines=3,
    )

    if start_page is None:
        logging.info("MD&A page number not found within strict 3-line window")
        return None

    logging.info(
        "MD&A ToC entry found (strict block): '%s' -> page %s",
        " ".join(raw_lines[i].get("line_text", "") for i in range(start_idx, end_idx + 1)),
        start_page,
    )
    return start_page


### 6. MD&A Boundary Detection : 

In [98]:
import re
import logging


def detect_mdna_boundaries(pages_text, toc_start_page=None):
    """Detect MD&A boundaries using STRICT Table of Contents (ToC) rules ONLY.

    Required behavior:
      - ToC is the only source of truth (first 5 pages only).
      - Treat ToC as STRUCTURAL BLOCKS: titles may be wrapped across lines.
      - When MD&A title is detected, search ONLY next 3 extracted lines for page number.
      - Deterministic alignment, no body-text inference; if not resolvable, SKIP.

    End page:
      - end_page = (next ToC section page - 1)
      - If next section page is missing, end_page = last page of document.

    Exclusions preserved:
      - Directors’ Report
      - Secretarial Audit (MR-3)
      - Corporate Information
      - Auditors’ Report
      - Corporate Governance

    Returns:
      (start_page, end_page) or (None, None)
    """

    max_page = len(pages_text)
    if max_page <= 0:
        logging.warning("Empty document; cannot detect MD&A boundaries")
        return None, None

    raw_lines = collect_toc_raw_lines(pages_text, max_pages=5)
    if not raw_lines:
        logging.warning("No ToC raw lines detected in the first 5 pages")
        return None, None

    mdna_title_re = re.compile(
        r"\bmanagement(?:\s*[’']?s)?\s+discussion(?:s)?\s+(?:and|&)\s+analysis(?:\s+report)?\b",
        re.IGNORECASE,
    )
    mdna_anchor_re = re.compile(r"\bmanagement\b", re.IGNORECASE)

    # Exclusion regexes (kept strict)
    directors_re = re.compile(r"\bdirectors\s*[’']?\s*report\b", re.IGNORECASE)
    secretarial_re = re.compile(r"\bsecretarial\s+audit\b", re.IGNORECASE)
    mr3_re = re.compile(r"\bform\s+mr\s*[-–]?\s*3\b|\bmr\s*[-–]?\s*3\b", re.IGNORECASE)
    corp_info_re = re.compile(r"\bcorporate\s+information\b", re.IGNORECASE)
    auditors_re = re.compile(r"\bauditors?\s*[’']?\s*report\b|\bindependent\s+auditor\b", re.IGNORECASE)
    corp_gov_re = re.compile(r"\bcorporate\s+governance\b", re.IGNORECASE)

    disallowed = [
        (directors_re, re.compile(r"\bdirectors\b", re.IGNORECASE)),
        (secretarial_re, re.compile(r"\bsecretarial\b", re.IGNORECASE)),
        (mr3_re, re.compile(r"\bmr\b|\bform\b", re.IGNORECASE)),
        (corp_info_re, re.compile(r"\bcorporate\b", re.IGNORECASE)),
        (auditors_re, re.compile(r"\bauditor\b|\bindependent\b", re.IGNORECASE)),
        (corp_gov_re, re.compile(r"\bgovernance\b|\bcorporate\b", re.IGNORECASE)),
    ]

    # --- 1) Find MD&A title as a structural block (up to 3 joined lines) ---
    mdna_start_idx, mdna_end_idx, mdna_block_text = find_title_block_strict(
        raw_lines,
        mdna_title_re,
        max_join_lines=3,
        anchor_re=mdna_anchor_re,
    )

    # --- 1a) Handle special case: MD&A is part of Directors' Report block ---
    if mdna_start_idx is None:
        logging.info("MD&A not found as standalone title; checking inside Directors' Report block")
        directors_anchor_re = re.compile(r"\bdirectors\b", re.IGNORECASE)
        dir_start_idx, dir_end_idx, dir_block_text = find_title_block_strict(
            raw_lines,
            directors_re,
            max_join_lines=3,
            anchor_re=directors_anchor_re,
        )

        if dir_block_text and mdna_title_re.search(dir_block_text):
            logging.info("MD&A title found inside Directors' Report block; using its boundaries")
            mdna_start_idx = dir_start_idx
            mdna_end_idx = dir_end_idx
            mdna_block_text = dir_block_text
        else:
            logging.warning("MD&A title block not found in ToC raw lines; skipping")
            return None, None

    # --- 2) STRICT page number association: page number after MD&A match, else next 3 lines ---
    mdna_page_details = resolve_page_number_for_title_block_strict_with_details(
        raw_lines,
        title_start_idx=mdna_start_idx,
        title_end_idx=mdna_end_idx,
        title_re=mdna_title_re,
        max_page=max_page,
        lookahead_lines=3,
    )

    if not mdna_page_details:
        logging.warning("MD&A start page not found within strict 3-line window; skipping")
        return None, None

    start_page = mdna_page_details["page"]

    # --- 3) Determine next section page (including same-line multi-column cases) ---
    def _next_page_number_in_same_line(line_text: str, after_pos: int, current_start_page: int):
        # Prefer numbers that occur after the current entry's page span (when ordering is preserved).
        res = _first_valid_page_number_after_pos_with_span(line_text, after_pos, max_page)
        if res:
            n, _, _ = res
            if n > current_start_page:
                return n

        # Fallback for multi-column merges where extraction order may be scrambled within the same line:
        # choose the smallest page number on the line that is greater than the current start page.
        candidates = []
        for m in _PAGE_INT_RE.finditer(line_text or ""):
            try:
                n = int(m.group(1))
            except ValueError:
                continue
            if current_start_page < n <= max_page:
                candidates.append(n)

        return min(candidates) if candidates else None

    def _next_section_start_page_after_line(after_line_idx: int, current_start_page: int):
        for j in range(after_line_idx + 1, len(raw_lines)):
            txt = raw_lines[j].get("line_text", "")
            if not re.search(r"[A-Za-z]", txt or ""):
                continue

            # Avoid treating the same MD&A title again
            if mdna_title_re.search(txt or ""):
                continue

            candidate = resolve_page_number_strict(raw_lines, j, max_page=max_page, lookahead_lines=3)
            if candidate is None:
                continue

            if candidate > current_start_page:
                return candidate

        return None

    same_line_idx = mdna_page_details["page_line_idx"]
    same_line_text = raw_lines[same_line_idx].get("line_text", "")
    same_line_next_page = _next_page_number_in_same_line(
        same_line_text,
        after_pos=mdna_page_details.get("page_span_end") or 0,
        current_start_page=start_page,
    )

    next_section_page = same_line_next_page
    if next_section_page is None:
        next_section_page = _next_section_start_page_after_line(mdna_end_idx, start_page)

    end_page = max_page if next_section_page is None else (next_section_page - 1)

    if end_page < start_page:
        logging.warning("Computed invalid MD&A range: start=%s end=%s; skipping", start_page, end_page)
        return None, None

    # --- 4) Exclusion ranges (handle same-line next-section; do not require resolving ALL exclusions) ---
    def _excluded_ranges():
        ranges = []

        for title_re, anchor_re in disallowed:
            # If MD&A was found inside the Directors' Report, don't treat Directors' Report as an exclusion
            if directors_re.pattern == title_re.pattern and mdna_title_re.search(mdna_block_text or ""):
                if directors_re.search(mdna_block_text or ""):
                    continue

            ex_start_idx, ex_end_idx, ex_block_text = find_title_block_strict(
                raw_lines,
                title_re,
                max_join_lines=3,
                anchor_re=anchor_re,
            )

            if ex_start_idx is None:
                continue

            ex_details = resolve_page_number_for_title_block_strict_with_details(
                raw_lines,
                title_start_idx=ex_start_idx,
                title_end_idx=ex_end_idx,
                title_re=title_re,
                max_page=max_page,
                lookahead_lines=3,
            )

            # If exclusion exists but cannot be aligned within strict window, we cannot
            # form a reliable range; skip enforcing that specific exclusion.
            if not ex_details:
                logging.warning("Excluded section found but page number not aligned; ignoring exclusion: %s", ex_block_text)
                continue

            ex_start_page = ex_details["page"]

            ex_same_line_idx = ex_details["page_line_idx"]
            ex_same_line_text = raw_lines[ex_same_line_idx].get("line_text", "")
            ex_same_line_next = _next_page_number_in_same_line(
                ex_same_line_text,
                after_pos=ex_details.get("page_span_end") or 0,
                current_start_page=ex_start_page,
            )

            ex_next_page = ex_same_line_next
            if ex_next_page is None:
                ex_next_page = _next_section_start_page_after_line(ex_end_idx, ex_start_page)

            # Critical safety for multi-column/boxed ToCs:
            # if an excluded section starts before MD&A (by page number), it must end no later
            # than the MD&A start page (as both are ToC-derived section starts), even if the
            # extracted line order is scrambled.
            if ex_start_page < start_page:
                if ex_next_page is None or start_page < ex_next_page:
                    ex_next_page = start_page

            ex_end_page = max_page if ex_next_page is None else (ex_next_page - 1)

            ranges.append(
                {
                    "title": ex_block_text,
                    "start": ex_start_page,
                    "end": ex_end_page,
                }
            )

        return ranges

    ex_ranges = _excluded_ranges()

    for r in ex_ranges:
        if r["start"] <= start_page <= r["end"]:
            logging.warning(
                "MD&A start page %s falls inside excluded section '%s' (%s-%s); skipping",
                start_page,
                r["title"],
                r["start"],
                r["end"],
            )
            return None, None

    logging.info("MD&A boundaries (STRICT ToC blocks): start=%s, end=%s", start_page, end_page)
    return start_page, end_page


### 7. MD&A Text Extraction (Boundary-Aware) : 

In [ ]:
def extract_mdna_text(pages_text, start_page, end_page):
    """Extract raw MD&A text from PDF pages using detected boundaries.

    Args:
        pages_text: List of page dictionaries from PDFInterface.get_pages_text(),
                    each like {"page_number": int, "text": str}
        start_page: 1-based start page number (inclusive)
        end_page: 1-based end page number (inclusive)

    Returns:
        str: Concatenated raw MD&A text (no cleaning), with double newlines
             inserted between pages.
    """

    included_text_chunks = []
    included_pages = 0

    for page in pages_text:
        page_no = page.get("page_number")
        if page_no is None:
            continue

        if start_page <= page_no <= end_page:
            included_pages += 1
            included_text_chunks.append(page.get("text", ""))

    mdna_text = "\n\n".join(included_text_chunks)

    logging.info("MD&A pages included: %d", included_pages)
    logging.info("Extracted MD&A text length (chars): %d", len(mdna_text))

    return mdna_text


# mdna_text = extract_mdna_text(pages, start_page, end_page)

# print("MD&A preview:\n")
# print(mdna_text[:1000])



2025-12-29 22:50:38,643 | INFO | root | MD&A pages included: 6
2025-12-29 22:50:38,655 | INFO | root | Extracted MD&A text length (chars): 18561


MD&A preview:

56 | AMTEK AUTO LIMITED
AMTEK AUTO LIMITED
MANAGEMENT DISCUSSION AND ANALYSIS REPORT
1.
GLOBAL ECONOMIC OVERVIEW
The overall performance of the global economy remained subdued through 2014, as well as into 2015. The world
economy grew 3.4% in 2014, impacted by a slowdown in many developing countries, which account for approximately
75% of the world economy. According to the International Monetary Fund (IMF), the global GDP growth rate is expected
to decline further by 30bps to 3.1% in 2015. A modest pickup in advanced economies and continued challenges in
emerging markets are the major factors behind these lower projections. The GDP growth for emerging markets and
developing countries in 2015 is expected to decline by 60bps to 4.0%, owing to weaker economic growth in the oil
exporting countries, a slowdown in China and expected negative growth in Brazil.
2.
INDIAN ECONOMIC OVERVIEW
During fiscal year 2015, the Indian economy started to show signs of a recovery after a pr

### 8. MD&A Text Cleaning & Artifact Removal: 

In [ ]:
def clean_mdna_text(raw_text):
    """Conservatively clean extracted MD&A text.

    What this does (conservative heuristics):
    - Removes obvious repeating headers/footers (e.g., 'Annual Report ...', standalone page numbers,
      and very header-like company-name lines when they appear repeatedly).
    - Removes obvious table artifacts (high digit-density lines, and separator-only lines).
    - Normalizes whitespace while preserving paragraph breaks.

    What this does NOT do:
    - Does not lowercase text
    - Does not remove punctuation
    - Does not change wording

    Args:
        raw_text: str

    Returns:
        str: cleaned MD&A text
    """

    if raw_text is None:
        raw_text = ""

    original_len = len(raw_text)

    # Split into lines to enable conservative line-based removals.
    lines = raw_text.splitlines()

    # Pre-compute line frequencies (normalized) to detect repeated headers/footers.
    def _norm_line_for_freq(line: str) -> str:
        return re.sub(r"\s+", " ", (line or "").strip())

    normalized_lines = [_norm_line_for_freq(ln) for ln in lines]
    freq = {}
    for nl in normalized_lines:
        if not nl:
            continue
        freq[nl] = freq.get(nl, 0) + 1

    annual_report_re = re.compile(r"^\s*Annual\s+Report(?:\s+\d{4}\s*[-–]\s*\d{2,4})?\s*$", re.IGNORECASE)
    standalone_page_no_re = re.compile(r"^\s*(?:page\s*)?\d{1,4}\s*$", re.IGNORECASE)
    separators_only_re = re.compile(r"^[\s\-_=*~•·\.\|:;,+/\\]+$")

    def _is_company_name_like(line: str) -> bool:
        # Conservative: only remove if it looks like a standalone header/footer line.
        s = (line or "").strip()
        if not s:
            return False
        if len(s) > 90:
            return False

        # Must contain common company suffixes.
        if not re.search(r"\b(LIMITED|LTD\.?|PVT\.?\s+LTD\.?|PRIVATE\s+LIMITED)\b", s, flags=re.IGNORECASE):
            return False

        # Must be mostly uppercase (typical header styling).
        letters = re.findall(r"[A-Za-z]", s)
        if not letters:
            return False
        upper_letters = sum(1 for ch in letters if ch.isupper())
        if upper_letters / len(letters) < 0.8:
            return False

        # Keep it short in words (header/footer line).
        if len(s.split()) > 10:
            return False

        return True

    def _is_high_numeric_density(line: str) -> bool:
        s = (line or "").strip()
        if len(s) < 12:
            return False
        # Density computed over non-space characters.
        compact = re.sub(r"\s+", "", s)
        if not compact:
            return False
        digits = sum(1 for ch in compact if ch.isdigit())
        return (digits / len(compact)) > 0.40

    cleaned_lines = []

    for raw_line, norm_line in zip(lines, normalized_lines):
        s = (raw_line or "").rstrip()
        sn = norm_line

        # Remove obvious separators/formatting-only lines.
        if sn and separators_only_re.match(sn):
            continue

        # Remove obvious page numbers (standalone).
        if sn and standalone_page_no_re.match(sn):
            continue

        # Remove 'Annual Report' headers/footers.
        if sn and annual_report_re.match(sn):
            continue

        # Remove frequent header/footer-like lines conservatively.
        # (Only if repeated AND header-ish AND not too long.)
        if sn and freq.get(sn, 0) >= 3:
            if _is_company_name_like(sn) or annual_report_re.match(sn) or standalone_page_no_re.match(sn):
                continue

        # Remove obvious table artifacts: high numeric density.
        if _is_high_numeric_density(sn):
            continue

        # Whitespace normalization inside the line.
        s = re.sub(r"[ \t]{2,}", " ", s).strip(" ")
        cleaned_lines.append(s)

    # Re-join with newlines and normalize paragraph spacing.
    cleaned_text = "\n".join(cleaned_lines)

    # Reduce 3+ consecutive newlines to at most 2.
    cleaned_text = re.sub(r"\n{3,}", "\n\n", cleaned_text)

    # Trim leading/trailing whitespace/newlines.
    cleaned_text = cleaned_text.strip()

    logging.info("MD&A text original length (chars): %d", original_len)
    logging.info("MD&A text cleaned length (chars): %d", len(cleaned_text))

    return cleaned_text

#sanity test: 
# cleaned_text = clean_mdna_text(mdna_text)

# print("Cleaned MD&A preview:\n")
# print(cleaned_text[:1200])


2025-12-29 15:06:51,628 | INFO | root | MD&A text original length (chars): 39415
2025-12-29 15:06:51,629 | INFO | root | MD&A text cleaned length (chars): 37792


Cleaned MD&A preview:

20 | AMTEK AUTO LIMITED
SECRETARIAL AUDIT REPORT
The Board has appointed M/s S. Khurana & Associates, Company Secretaries, to conduct Secretarial Audit for the financial
year 2015-16. The Secretarial Audit Report for the financial year ended March 31, 2016 is annexed herewith marked as
Annexure - I to this Report. The Secretarial Audit Report does not contain any qualification, reservation or adverse remark.
As per the directive of Securities and Exchange Board of India, M/s S. Khurana & Associates Company Secretaries, New
Delhi, undertook the Reconciliation of Share Capital Audit on a quarterly basis. The purpose of the audit is to reconcile the
total number of shares held in National Securities Depository Limited (NSDL), Central Depository Services (India) Limited
(CDSL) and in physical form with the respect to admitted, issued and paid up capital of the Company.
CORPORATE GOVERNANCE
The Company is committed to maintain high standards of Corporate Governance an

### 9. MD&A Quality Verification & Validation Metrics : 

In [70]:
def verify_mdna_quality(mdna_text, pages_match_toc: bool):
    """Verify extracted/cleaned MD&A text quality using STRICT semantics.

    quality_passed MUST be TRUE only if:
      - Extracted pages match the ToC-declared MD&A range exactly (pages_match_toc=True)
      - Text contains at least 2 MD&A-specific phrases (case-insensitive)

    Guardrail:
      - Audit / Corporate / Director section signals must ALWAYS fail quality.

    Returns:
        dict with keys:
          - word_count (int)
          - narrative_density (float)
          - keyword_hits (list[str])
          - quality_passed (bool)
    """

    if mdna_text is None:
        mdna_text = ""

    # a) Word count
    words = re.findall(r"\b\w+\b", mdna_text)
    word_count = len(words)

    # b) Narrative density (alphabetic chars / total chars)
    total_chars = len(mdna_text)
    alpha_chars = sum(1 for ch in mdna_text if ch.isalpha())
    narrative_density = (alpha_chars / total_chars) if total_chars else 0.0

    lower_text = mdna_text.lower()

    # c) Required MD&A phrases
    mdna_phrases = [
        "industry outlook",
        "opportunities and threats",
        "risk management",
        "future outlook",
        "segment performance",
        "global economy",
    ]

    keyword_hits = [p for p in mdna_phrases if p in lower_text]

    # Disqualifiers: MUST always fail
    disqualifiers = [
        "independent auditor",
        "auditor's report",
        "auditors' report",
        "auditors report",
        "secretarial audit",
        "form mr-3",
        "mr-3",
        "corporate information",
        "corporate governance",
        "directors' report",
        "director's report",
        "directors report",
    ]

    disqualifier_hit = any(bad in lower_text for bad in disqualifiers)

    criteria_pages_match = bool(pages_match_toc)
    criteria_phrases = len(keyword_hits) >= 2

    quality_passed = bool(criteria_pages_match and criteria_phrases and (not disqualifier_hit))

    logging.info("MD&A quality — word_count: %d", word_count)
    logging.info("MD&A quality — narrative_density: %.4f", narrative_density)
    logging.info("MD&A quality — keyword_hits (%d): %s", len(keyword_hits), keyword_hits)
    logging.info("MD&A quality — pages_match_toc: %s", criteria_pages_match)
    logging.info("MD&A quality — disqualifier_hit: %s", disqualifier_hit)

    if quality_passed:
        logging.info("MD&A quality PASSED")
    else:
        logging.warning(
            "MD&A quality FLAGGED (pages_match_toc=%s, phrases_ok=%s, disqualifier_hit=%s)",
            criteria_pages_match,
            criteria_phrases,
            disqualifier_hit,
        )

    return {
        "word_count": word_count,
        "narrative_density": float(narrative_density),
        "keyword_hits": keyword_hits,
        "quality_passed": quality_passed,
    }


In [103]:
from pathlib import Path
from datetime import datetime

# -----------------------------
# End-to-end MD&A Extraction Pipeline (STRICT ToC-Only Boundaries)
# -----------------------------
PDF_ROOT = Path("../data/pdfs")

# Prefer the configured OUTPUT_DIR if present; otherwise default to ../output
try:
    output_dir = OUTPUT_DIR
except NameError:
    output_dir = Path("../output")

output_dir.mkdir(parents=True, exist_ok=True)

pdf_paths = sorted(PDF_ROOT.rglob("*.pdf"))
logging.info("Discovered %d PDFs under %s", len(pdf_paths), PDF_ROOT)

results = []

attempted = 0
succeeded = 0
skipped_no_boundaries = 0
failed = 0
passed_quality = 0
flagged_quality = 0

for idx, pdf_path in enumerate(pdf_paths, start=1):
    attempted += 1
    company_folder = pdf_path.parent.name

    logging.info("(%d/%d) Processing: %s/%s", idx, len(pdf_paths), company_folder, pdf_path.name)

    pdf = None
    try:
        pdf = PDFInterface(pdf_path)
        pages = pdf.get_pages_text()

        company_name = extract_company_name(pages)
        financial_year = extract_financial_year(pages)

        # STRICT ToC-only boundaries: if not deterministically resolvable from ToC lines, skip.
        start_page, end_page = detect_mdna_boundaries(pages_text=pages, toc_start_page=None)

        if not start_page or not end_page:
            skipped_no_boundaries += 1
            logging.warning("Skipping (MD&A boundaries not determinable via STRICT ToC rules): %s", pdf_path.name)
            continue

        raw_mdna_text = extract_mdna_text(pages_text=pages, start_page=start_page, end_page=end_page)
        cleaned_mdna_text = clean_mdna_text(raw_mdna_text)

        # STRICT: pages must match the ToC-declared MD&A range exactly
        expected_pages = end_page - start_page + 1
        actual_pages = sum(
            1
            for p in pages
            if isinstance(p.get("page_number"), int) and start_page <= p.get("page_number") <= end_page
        )
        pages_match_toc = (expected_pages == actual_pages)

        quality_report = verify_mdna_quality(cleaned_mdna_text, pages_match_toc=pages_match_toc)

        results.append(
            {
                "company_folder": company_folder,
                "company_name": company_name,
                "report_file": pdf_path.name,
                "financial_year": financial_year,
                "mdna_text": cleaned_mdna_text,
                "mdna_word_count": quality_report.get("word_count"),
                "narrative_density": quality_report.get("narrative_density"),
                "keyword_hits": quality_report.get("keyword_hits"),
                "quality_passed": quality_report.get("quality_passed"),
            }
        )

        succeeded += 1
        if quality_report.get("quality_passed"):
            passed_quality += 1
        else:
            flagged_quality += 1

    except Exception as exc:
        failed += 1
        logging.exception("Failed processing %s: %s", pdf_path, exc)

    finally:
        try:
            if pdf is not None:
                pdf.close()
        except Exception:
            # Ignore close errors to keep the pipeline robust.
            pass

logging.info("Total PDFs discovered: %d", len(pdf_paths))
logging.info("Total PDFs attempted: %d", attempted)
logging.info("Total PDFs succeeded: %d", succeeded)
logging.info("Total PDFs skipped (no STRICT ToC boundaries): %d", skipped_no_boundaries)
logging.info("Total PDFs failed: %d", failed)
logging.info("Quality checks passed: %d", passed_quality)
logging.info("Quality checks flagged: %d", flagged_quality)

results_df = pd.DataFrame(results)

csv_path = output_dir / "mdna_extracted.csv"
xlsx_path = output_dir / "mdna_extracted.xlsx"

# Safe writes: Windows may lock files if they're open in Excel.
run_stamp = datetime.now().strftime("%Y%m%d_%H%M%S")

try:
    results_df.to_csv(csv_path, index=False)
    logging.info("Wrote CSV: %s", csv_path)
except PermissionError as exc:
    fallback_csv = output_dir / f"mdna_extracted_{run_stamp}.csv"
    logging.warning("CSV locked (%s). Writing fallback CSV: %s", exc, fallback_csv)
    results_df.to_csv(fallback_csv, index=False)

# Excel export needs sanitization because openpyxl rejects some control characters.
_illegal_excel_chars_re = re.compile(r"[\x00-\x08\x0B\x0C\x0E-\x1F]")

def _sanitize_for_excel(value):
    if value is None:
        return value
    if isinstance(value, str):
        return _illegal_excel_chars_re.sub("", value)
    return value

excel_df = results_df.copy()
if "keyword_hits" in excel_df.columns:
    excel_df["keyword_hits"] = excel_df["keyword_hits"].apply(
        lambda x: ", ".join(x) if isinstance(x, list) else x
    )

for col in excel_df.columns:
    if excel_df[col].dtype == "object":
        excel_df[col] = excel_df[col].apply(_sanitize_for_excel)

try:
    excel_df.to_excel(xlsx_path, index=False)
    logging.info("Wrote Excel: %s", xlsx_path)
except PermissionError as exc:
    fallback_xlsx = output_dir / f"mdna_extracted_{run_stamp}.xlsx"
    logging.warning("Excel locked (%s). Writing fallback Excel: %s", exc, fallback_xlsx)
    excel_df.to_excel(fallback_xlsx, index=False)
except Exception as exc:
    logging.exception("Failed to write Excel output: %s", exc)

results_df

2025-12-30 00:04:04,490 | INFO | root | Discovered 14 PDFs under ..\data\pdfs


2025-12-30 00:04:04,492 | INFO | root | (1/14) Processing: Alcheimist/5267070319.pdf
2025-12-30 00:04:04,496 | INFO | root | Loaded PDF: 5267070319.pdf
2025-12-30 00:04:04,891 | INFO | root | Extracted company name: You are requested to take the above mentioned information on your records.
For Alchemist Limited
2025-12-30 00:04:04,892 | INFO | root | Extracted financial year: 2018-19
2025-12-30 00:04:04,893 | WARNING | root | Excluded section found but page number not aligned; ignoring exclusion: DIRECTORS’ REPORT
2025-12-30 00:04:04,895 | INFO | root | MD&A boundaries (STRICT ToC blocks): start=23, end=100
2025-12-30 00:04:04,895 | INFO | root | MD&A pages included: 78
2025-12-30 00:04:04,896 | INFO | root | Extracted MD&A text length (chars): 243745
2025-12-30 00:04:04,952 | INFO | root | MD&A text original length (chars): 243745
2025-12-30 00:04:04,953 | INFO | root | MD&A text cleaned length (chars): 229396
2025-12-30 00:04:04,983 | INFO | root | MD&A quality — word_count: 37325
20

,company_folder,company_name,report_file,financial_year,mdna_text,mdna_word_count,narrative_density,keyword_hits,quality_passed
0,Alcheimist,You are requested to take the above mentioned ...,5267070319.pdf,2018-19,V.\nINDEBTEDNESS - Indebtedness of the Company...,37325,0.770414,"[risk management, segment performance]",False
1,Alcheimist,Bankers\nPunjab National Bank\nBank Of india\n...,67050526707.pdf,2019-20,Annexure VI\nMANAGEMENT DISCUSSION AND ANALYSI...,2740,0.816677,[],False
2,ALok,None,5210700315.pdf,None,PRIMED\nTO UNLOCK VALUE\n28th Annual Report | ...,16737,0.750986,[],False
3,ALok,None,5210700316.pdf,2015-16,Innovative Textile Solutions 43\nForm No. MR-3...,11815,0.755314,[],False
4,ALok,None,5210700317.pdf,2016-17,"As mentioned earlier, the overall performance ...",368,0.820254,[],False
5,ALok,None,5210700318.pdf,2017-18,(vi)\nThe following laws are applicable to the...,9254,0.730728,"[risk management, global economy]",False
6,Amit_spinning,AMIT SPINNING INDUSTRIES LIMITED,5210760315.pdf,2014-15,DIRECTORS’ REPORT\nYour Directors have great p...,5952,0.774771,"[risk management, future outlook]",False
7,Amit_spinning,JM Financial Asset Reconstruction Company Pvt....,5210760316.pdf,2015-16,"SUBSIDIARY COMPANIES, JOINT VENTURES AND ASSOC...",5666,0.757234,[risk management],False
8,Amtek,Bankers\nCorporation Bank\nAndhra Bank\nIndian...,5200770316.pdf,2015-16,62 | AMTEK AUTO LIMITED\nAUDITORS’ REPORT ON C...,4742,0.801443,"[risk management, global economy]",False
9,Amtek,Bankers\nCorporation Bank\nAndhra Bank\nIndian...,5200770317.pdf,2016-17,ANNUAL REPORT 2016-17 | 63\nCEO/CFO CERTIFICAT...,4994,0.802536,[],False


In [101]:
# Quick summary of how many rows were extracted
print("\nPipeline output summary")
print("- results_df shape:", results_df.shape)
print("- quality passed:", int((results_df["quality_passed"] == True).sum()) if "quality_passed" in results_df.columns else "N/A")
print("- quality flagged:", int((results_df["quality_passed"] == False).sum()) if "quality_passed" in results_df.columns else "N/A")



Pipeline output summary
- results_df shape: (12, 9)
- quality passed: 0
- quality flagged: 12


### Test cell — MD&A Boundary Detection 

In [99]:
from pathlib import Path

PDF_ROOT = Path("../data/pdfs")

# Pick ONE representative PDF per company
test_pdfs = {}
for pdf in PDF_ROOT.rglob("*.pdf"):
    company = pdf.parent.name
    if company not in test_pdfs:
        test_pdfs[company] = pdf

print("Testing MD&A boundary detection on sample PDFs:\n")

for company, pdf_path in test_pdfs.items():
    print("=" * 70) 
    print(f"Company Folder : {company}")
    print(f"PDF File       : {pdf_path.name}")

    pdf = PDFInterface(pdf_path)
    pages = pdf.get_pages_text()

    start_page, end_page = detect_mdna_boundaries(
        pages_text=pages,
        toc_start_page=None
    )

    print(f"Detected MD&A Start Page: {start_page}")
    print(f"Detected MD&A End Page  : {end_page}")

    if start_page and end_page:
        assert start_page <= end_page, "Start page must be strictly before end page"
        assert 1 <= start_page <= len(pages), "Start page out of range"
        assert 1 <= end_page <= len(pages), "End page out of range"
        print("✔ Boundary detection looks valid")
    else:
        print("⚠ MD&A boundaries not detected (may require fallback logic)")

print("\nBoundary detection test completed.")


2025-12-30 00:01:47,457 | INFO | root | Loaded PDF: 5267070319.pdf


Testing MD&A boundary detection on sample PDFs:

Company Folder : Alcheimist
PDF File       : 5267070319.pdf


2025-12-30 00:01:47,845 | WARNING | root | Excluded section found but page number not aligned; ignoring exclusion: DIRECTORS’ REPORT
2025-12-30 00:01:47,847 | INFO | root | MD&A boundaries (STRICT ToC blocks): start=23, end=100
2025-12-30 00:01:47,853 | INFO | root | Loaded PDF: 5210700315.pdf


Detected MD&A Start Page: 23
Detected MD&A End Page  : 100
✔ Boundary detection looks valid
Company Folder : ALok
PDF File       : 5210700315.pdf


2025-12-30 00:01:48,463 | INFO | root | MD&A boundaries (STRICT ToC blocks): start=51, end=82
2025-12-30 00:01:48,466 | INFO | root | Loaded PDF: 5210760315.pdf
2025-12-30 00:01:48,627 | INFO | root | MD&A boundaries (STRICT ToC blocks): start=8, end=15
2025-12-30 00:01:48,630 | INFO | root | Loaded PDF: 5200770316.pdf


Detected MD&A Start Page: 51
Detected MD&A End Page  : 82
✔ Boundary detection looks valid
Company Folder : Amit_spinning
PDF File       : 5210760315.pdf
Detected MD&A Start Page: 8
Detected MD&A End Page  : 15
✔ Boundary detection looks valid
Company Folder : Amtek
PDF File       : 5200770316.pdf


2025-12-30 00:01:48,912 | INFO | root | MD&A boundaries (STRICT ToC blocks): start=63, end=71


Detected MD&A Start Page: 63
Detected MD&A End Page  : 71
✔ Boundary detection looks valid

Boundary detection test completed.


### Sanity check  after cell 4: 

In [102]:
from pathlib import Path
import os

# ---- CONFIG ----
PDF_ROOT = Path("../data/pdfs")

print(f"Current working directory: {os.getcwd()}")
print(f"PDF_ROOT: {PDF_ROOT}")
print(f"PDF_ROOT exists: {PDF_ROOT.exists()}")
print(f"PDF_ROOT resolved: {PDF_ROOT.resolve()}")

# ---- STEP 1: Discover PDFs ----
pdf_files = list(PDF_ROOT.rglob("*.pdf"))

print(f"Total PDFs found: {len(pdf_files)}")

if not pdf_files:
    print("No PDFs found in ../data/pdfs directory. Please add PDF files to test the pipeline.")
    print("Skipping sanity checks.")
else:
    # Pick one PDF from each company (if available)
    sample_pdfs = {}
    for pdf in pdf_files:
        company = pdf.parent.name
        if company not in sample_pdfs:
            sample_pdfs[company] = pdf

    print("\nSample PDFs selected for testing:")
    for company, pdf in sample_pdfs.items():
        print(f"- {company}: {pdf.name}")

    # ---- STEP 2: Test PDFInterface + Metadata Extraction ----
    print("\n--- Running Sanity Checks ---\n")

    for company, pdf_path in sample_pdfs.items():
        print(f"Testing company: {company}")
        print(f"PDF: {pdf_path.name}")

        pdf = PDFInterface(pdf_path)
        pages = pdf.get_pages_text()

        print("Pages extracted:", len(pages))
        assert len(pages) > 0, "No pages extracted!"

        # Metadata extraction
        extracted_company = extract_company_name(pages)
        extracted_year = extract_financial_year(pages)

        print("Extracted Company Name:", extracted_company)
        print("Extracted Financial Year:", extracted_year)

        print("-" * 50)

    print("\nSanity check completed successfully.")

2025-12-30 00:02:52,892 | INFO | root | Loaded PDF: 5267070319.pdf


Current working directory: c:\Users\LOQ\Desktop\SPJIMR\mdna_extraction_project\notebooks
PDF_ROOT: ..\data\pdfs
PDF_ROOT exists: True
PDF_ROOT resolved: C:\Users\LOQ\Desktop\SPJIMR\mdna_extraction_project\data\pdfs
Total PDFs found: 14

Sample PDFs selected for testing:
- Alcheimist: 5267070319.pdf
- ALok: 5210700315.pdf
- Amit_spinning: 5210760315.pdf
- Amtek: 5200770316.pdf

--- Running Sanity Checks ---

Testing company: Alcheimist
PDF: 5267070319.pdf


2025-12-30 00:02:53,310 | INFO | root | Extracted company name: You are requested to take the above mentioned information on your records.
For Alchemist Limited
2025-12-30 00:02:53,311 | INFO | root | Extracted financial year: 2018-19
2025-12-30 00:02:53,316 | INFO | root | Loaded PDF: 5210700315.pdf


Pages extracted: 145
Extracted Company Name: You are requested to take the above mentioned information on your records.
For Alchemist Limited
Extracted Financial Year: 2018-19
--------------------------------------------------
Testing company: ALok
PDF: 5210700315.pdf


2025-12-30 00:02:53,995 | INFO | root | Company name not found in first 3 pages
2025-12-30 00:02:53,996 | INFO | root | Financial year not found in first 5 pages
2025-12-30 00:02:53,999 | INFO | root | Loaded PDF: 5210760315.pdf


Pages extracted: 208
Extracted Company Name: None
Extracted Financial Year: None
--------------------------------------------------
Testing company: Amit_spinning
PDF: 5210760315.pdf


2025-12-30 00:02:54,216 | INFO | root | Extracted company name: AMIT SPINNING INDUSTRIES LIMITED
2025-12-30 00:02:54,217 | INFO | root | Extracted financial year: 2014-15
2025-12-30 00:02:54,220 | INFO | root | Loaded PDF: 5200770316.pdf


Pages extracted: 46
Extracted Company Name: AMIT SPINNING INDUSTRIES LIMITED
Extracted Financial Year: 2014-15
--------------------------------------------------
Testing company: Amtek
PDF: 5200770316.pdf


2025-12-30 00:02:54,522 | INFO | root | Extracted company name: Bankers
Corporation Bank
Andhra Bank
Indian Overseas Bank
IDBI Bank
Registrar & Share Transfer Agent
Beetal Financial & Computer
Services Pvt. Ltd.
2025-12-30 00:02:54,522 | INFO | root | Extracted financial year: 2015-16


Pages extracted: 145
Extracted Company Name: Bankers
Corporation Bank
Andhra Bank
Indian Overseas Bank
IDBI Bank
Registrar & Share Transfer Agent
Beetal Financial & Computer
Services Pvt. Ltd.
Extracted Financial Year: 2015-16
--------------------------------------------------

Sanity check completed successfully.
